In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm
torch.manual_seed(1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cpu = torch.device('cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [38]:
import torch.nn as nn

class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.5):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.mixnet = timm.create_model("mixnet_m", pretrained=True)
        for param in self.mixnet.parameters():
                param.requires_grad = True
            
        self.lstm = nn.LSTM(1000, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.75)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_dim, 32)
#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(32, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size, seqlen, c, h, w = x.size()
        x = x.reshape(batch_size*seqlen, c, h, w).float()
        x = self.mixnet(x)
        x = x.reshape(batch_size, seqlen, x.shape[1])
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
#         out = self.batchnorm(out)
        out = self.fc1(out)
        out = self.elu(out)
#         out = self.fc2(out)
#         out = self.elu(out)
#         out = self.fc3(out)
#         out = self.elu(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [6]:
'data_images/000metadata_part_', str(p), '.pt'

('data_images/000metadata_part_', '0', '.pt')

In [21]:
for p in np.arange(0,8):
    if p == 0:
        metadata = torch.load('data_images/000metadata_part_' + str(p) + '.pt', map_location = cpu)
    else:
        metadata_p = torch.load('data_images/000metadata_part_' + str(p) + '.pt', map_location = cpu)
        metadata = pd.concat([metadata, metadata_p])

In [22]:
metadata.n_face.value_counts()

1    13648
2     2042
0      667
Name: n_face, dtype: int64

So far, we have 13.6K videos with 1 face consistently, then some with 2 and some with none.

In [23]:
metadata.head()

,label,original,split,n_face
owxbbpjpch.mp4,FAKE,wynotylpnm.mp4,train,1
vpmyeepbep.mp4,REAL,NaN,train,1
fzvpbrzssi.mp4,REAL,NaN,train,1
htorvhbcae.mp4,FAKE,wclvkepakb.mp4,train,1
fckxaqjbxk.mp4,FAKE,vpmyeepbep.mp4,train,1


In [24]:
X = np.array(metadata.index[metadata.n_face == 1])
Y = np.array(1 * (metadata.label[metadata.n_face == 1] == 'REAL'))

In [28]:
X[0:5]

array(['owxbbpjpch.mp4', 'vpmyeepbep.mp4', 'fzvpbrzssi.mp4',
       'htorvhbcae.mp4', 'fckxaqjbxk.mp4'], dtype=object)

In [29]:
Y[0:5]

array([0, 1, 1, 0, 0])

In [30]:
len(X)

13648

In [31]:
len(Y)

13648

In [59]:
from torch.utils.data import TensorDataset, DataLoader
dataset = TensorDataset(torch.from_numpy(np.arange(0,len(X))), torch.from_numpy(Y))
train_data, val_data = torch.utils.data.random_split(dataset, [12000, 1648])
train_batch_size = 5
val_batch_size = 2
train_loader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=val_batch_size)

In [9]:

# class Identity(nn.Module):
#     def __init__(self):
#         super(Identity, self).__init__()
#     def forward(self, x):
#         return x

# model_ft = timm.create_model("mixnet_m", pretrained=True)
# model_ft.classifier = Identity()
# batch, seqlen, c, h, w =  X.shape
# X = X.reshape(batch*seqlen, c, h, w) # .shape
# out = model_ft(X)
# out.shape
# out = out.reshape(batch, seqlen, out.shape[1])
# out.shape

In [39]:
input_size = 512
output_size = 1
hidden_dim = 64
n_layers = 5

model = DFDCNet(input_size, output_size, hidden_dim, n_layers)
model.to(device)
train_criterion = nn.BCELoss()
val_criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mixnet_m-4647fc68.pth" to /root/.cache/torch/checkpoints/mixnet_m-4647fc68.pth


DFDCNet(
  (mixnet): EfficientNet(
    (conv_stem): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=24, bias=False)
          (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv_pw): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (1): Sequential(
        (0): InvertedResidual(
          (conv_pw): MixedConv2d(
            (0): Conv2d(12, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): Conv2d(1

In [40]:
# scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=500, verbose=True)


In [54]:
epochs = 5
counter = 0
print_every = 1
clip = .5
valid_loss_min = .5
val_loss = torch.tensor(np.Inf)
model.train()
for i in range(epochs):
    h = model.init_hidden(train_batch_size)
    
    for inputs, labels in train_loader:
        inputs = torch.stack([torch.load('data_images/1face_X_'+ name + '.pt') for name in X[inputs]])
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = train_criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(val_batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                inp = torch.stack([torch.load('data_images/1face_X_'+ name + '.pt') for name in X[inp]])
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = val_criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './model_1face_unfroze.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
    scheduler.step(val_loss.item())


FileNotFoundError: [Errno 2] No such file or directory: 'data_images/1face_X_t.pt'

# Reference
* https://github.com/ronghanghu/pytorch-gve-lrcn/blob/master/models/pretrained_models.py